In [2]:
import pandas as pd
import numpy  as np
import re

In [3]:
movie_titles = open("movie_titles.txt").read()
netflix_training = open("TrainingRatings.txt").read()
netflix_testing = open("TestingRatings.txt").read()

In [15]:
# Tranform text files into DataFrames
movie_titles_df     = pd.DataFrame([i.split(',',2) for i in re.sub('[\r]', '', movie_titles).split('\n')][:-1])
netflix_training_df = pd.DataFrame([i.split(',',2) for i in re.sub('[\r]', '', netflix_training).split('\n')[:-1]])
netflix_testing_df  = pd.DataFrame([i.split(',',2) for i in re.sub('[\r]', '', netflix_testing).split('\n')[:-1]])
movie_titles_df.columns = ['MovieID', 'YearOfRelease', 'Title']
netflix_testing_df.columns = netflix_training_df.columns = ['MovieID', 'UserID', 'Rating']

In [34]:
# Transform all relevant string values to int
movie_titles_df['MovieID'] = movie_titles_df['MovieID'].astype(int)

netflix_testing_df['MovieID'] = netflix_testing_df['MovieID'].astype(int)
netflix_testing_df['UserID'] = netflix_testing_df['UserID'].astype(int)
netflix_testing_df['Rating'] = netflix_testing_df['Rating'].astype(np.float64)

netflix_training_df['MovieID'] = netflix_training_df['MovieID'].astype(int)
netflix_training_df['UserID'] = netflix_training_df['UserID'].astype(int)
netflix_training_df['Rating'] = netflix_training_df['Rating'].astype(np.float64)

In [176]:
def Average_Vote(votes):
    return np.sum(votes) / len(votes)

def Predict_User_Vote(train_df, user_id, movie_id):
    user = train_df[train_df['UserID'] == user_id]
    user_avg = Average_Vote(user['Rating'])
    other_users = train_df[train_df['UserID'] != user_id]
    relevant_users = other_users[other_users['MovieID'] == movie_id]['UserID'].unique()
    #print relevant_users
    k = 1 #Normalizing factor, find out how to do this
    score = np.sum([Get_Correlation_Prediction(other_users, user, user_avg, u, movie_id) for u in relevant_users])
    return (Average_Vote(user['Rating']) + k * score).round()

def Get_Correlation_Prediction(other_users, user, user_avg, other_user_id, movie_id):
    other_user     = other_users[other_users['UserID'] == other_user_id]
    other_user_avg = Average_Vote(other_user['Rating'])
    vote_weight    = other_user[other_user['MovieID'] == movie_id]['Rating'].iloc[0] - other_user_avg
    return Correlation_Weight_Between_User(user, user_avg, other_user, other_user_avg) * vote_weight

def Correlation_Weight_Between_User(user, user_avg, other_user, other_user_avg):
    other_user_avg = Average_Vote(other_user['Rating'])
    common_movies = set(user['MovieID']).intersection(set(other_user['MovieID']))
    numerator   = np.sum([Vote_Minus_Mean(user, user_avg, movie_id) *
                        Vote_Minus_Mean(other_user, other_user_avg, movie_id)
                        for movie_id in common_movies])
    denominator = np.sqrt( np.sum([Vote_Minus_Mean(user, user_avg, movie_id)**2 
                                   for movie_id in common_movies]) *
                           np.sum([Vote_Minus_Mean(other_user, other_user_avg, movie_id)**2 
                                   for movie_id in common_movies]))
    return numerator / denominator
    
def Vote_Minus_Mean(user, user_avg, movie_id):
    return user[user['MovieID'] == movie_id]['Rating'].iloc[0] - user_avg

In [177]:
Predict_User_Vote(netflix_training_df, 1744889, 8)

/home/daniel/.conda/envs/csgrads1/lib/python2.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in double_scalars


nan

In [153]:
netflix_training_df

,MovieID,UserID,Rating
0,8,1744889,1.0
1,8,1395430,2.0
2,8,1205593,4.0
3,8,1488844,4.0
4,8,1447354,1.0
5,8,306466,4.0
6,8,1331154,4.0
7,8,1818178,3.0
8,8,991725,4.0
9,8,1987434,4.0
